In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import keras 
from keras.layers import Dense, Conv2D, Flatten, MaxPool2D,BatchNormalization
from keras.models import Sequential
from keras.applications.vgg16 import VGG16

In [2]:
conv_part = VGG16(
    include_top=False, # not using dense part
    weights="imagenet",
    input_shape=(256,256,3)
)

58889256/58889256 [==============================] - 267s 5us/step


In [3]:
conv_part.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 256, 256, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 256, 256, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 128, 128, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 128, 128, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 128, 128, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 64, 64, 128)       0     

In [4]:
model = Sequential()
model.add(conv_part)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 8, 8, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 32768)             0         
                                                                 
 dense (Dense)               (None, 256)               8388864   
                                                                 
 dense_1 (Dense)             (None, 1)                 257       
                                                                 
Total params: 23103809 (88.13 MB)
Trainable params: 23103809 (88.13 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### Frezing the Conv Part

In [7]:
conv_part.trainable = False

### Using Generators for loading data from folder

In [8]:
# using generators

train_ds = tf.keras.utils.image_dataset_from_directory(
    directory= 'D:\#DATA Science\Deep Learning\Deep Learning Practice\dogs_vs_cats\\train',
    labels='inferred',
    label_mode='int',
    batch_size=32,
    image_size=(256,256)

)

test_ds = tf.keras.utils.image_dataset_from_directory(
    directory= 'D:\#DATA Science\Deep Learning\Deep Learning Practice\dogs_vs_cats\\test',
    labels='inferred',
    label_mode='int',
    batch_size=32,
    image_size=(256,256)

)



Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.


### Normalization of Images

In [9]:
#normalization
def normalize(image,label):
    image = tf.cast(image/255. , tf.float64)
    return image, label

train_ds = train_ds.map(normalize)
test_ds = test_ds.map(normalize)

In [11]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(train_ds, validation_data=test_ds, epochs=10)